<a href="https://colab.research.google.com/github/Ut14/AI-Powered-Fitness-Assistant/blob/main/Gradio_fitness.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install gradio langchain langgraph langchain_huggingface \
langchain_community cassio cassandra-driver \
chromadb transformers accelerate tiktoken \
langchain_groq google-search-results

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 152.4/152.4 kB 16.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 104.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.7/45.7 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.9/3.9 MB 83.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.3/19.3 MB 86.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 29.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/130.2 kB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
import json
import os
import uuid

PROFILE_PATH = "user_profile.json"

def initialize_profile():
    if not os.path.exists(PROFILE_PATH):
        with open(PROFILE_PATH, "w") as f:
            json.dump({"age": None, "height": None, "weight": None, "goal": None}, f, indent=4)

def update_user_profile(profile_data):
    with open(PROFILE_PATH, "w") as f:
        json.dump(profile_data, f, indent=4)

def load_user_profile():
    with open(PROFILE_PATH, "r") as f:
        return json.load(f)

initialize_profile()


In [3]:
import cassio
from langchain.vectorstores.cassandra import Cassandra
from langchain_huggingface import HuggingFaceEmbeddings

ASTRA_DB_APPLICATION_TOKEN = "AstraCS:LCogOZmNAWvKZThNEDeAEMkN:bfebe772ee835275b0c77949110a374e0a5ed371bba8b3040760aeb9f0724feb"
ASTRA_DB_ID ="b7761811-cebd-4761-b1ef-c2009d1f02c9"

cassio.init(token=ASTRA_DB_APPLICATION_TOKEN, database_id=ASTRA_DB_ID)

embedding_model = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
astra_vector_store = Cassandra(
    embedding=embedding_model,
    table_name="fitness_notes",
    session=None,
    keyspace=None
)

retriever = astra_vector_store.as_retriever()


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [4]:
from langchain.schema import Document
from datetime import datetime

def add_note_to_vectorstore(note: str):
    profile = load_user_profile()
    note_id = str(uuid.uuid4())
    profile["timestamp"] = str(datetime.now())
    profile["note_id"] = note_id
    doc = Document(page_content=note, metadata=profile)
    astra_vector_store.add_documents([doc])
    return "✅ Note added."

def fetch_notes(query: str):
    documents = retriever.invoke(query)
    return [doc.page_content for doc in documents]

def get_all_notes():
    results = astra_vector_store.similarity_search("", k=100)
    return [(doc.page_content, doc.metadata.get("timestamp", ""), doc.metadata.get("note_id", "")) for doc in results]

def get_notes_display_list():
    all_notes = get_all_notes()
    display = [f"{i+1}. {n[0][:80]}... (added on {n[1]})" for i, n in enumerate(all_notes)]
    note_ids = [n[2] for n in all_notes]
    return display, note_ids

def delete_selected_notes(selected_notes_display):
    if not selected_notes_display:
        return "⚠️ No notes selected."

    display_list, note_ids = get_notes_display_list()
    to_delete_indices = [display_list.index(sn) for sn in selected_notes_display]
    ids_to_delete = [note_ids[i] for i in to_delete_indices]
    astra_vector_store.delete(ids_to_delete)
    return f"🗑️ Deleted {len(ids_to_delete)} note(s)."

In [5]:
from langchain_community.utilities import SerpAPIWrapper
import os
from google.colab import userdata

serp_api_key = userdata.get('serp_api_key')
os.environ["SERPAPI_API_KEY"] = serp_api_key
search = SerpAPIWrapper()

def search_web(query: str):
    results = search.results(query)
    if not results.get("organic_results"):
        return "No results."
    return "\n".join([f"🔹 {r.get('title')}\n🔗 {r.get('link')}" for r in results["organic_results"][:3]])


In [6]:
from langchain_groq import ChatGroq
from langchain_core.prompts import PromptTemplate
from google.colab import userdata

groq_api_key = userdata.get("groq_api_key")
llm = ChatGroq(groq_api_key=groq_api_key, model_name="llama3-70b-8192")

router_prompt = PromptTemplate.from_template("""
You are an expert decision maker.

You are given:
- The user's profile
- Their personal fitness notes
- Their current question

Decide whether you have enough information to answer **without web search**.

If insufficient, say: **"USE_WEB"**, else say: **"NO_WEB"**

{context}
""")

def should_use_web(context: str) -> bool:
    router_decision = llm.invoke(router_prompt.format(context=context)).content.strip()
    return router_decision == "USE_WEB"


In [7]:
from langgraph.graph import StateGraph, START, END
from typing_extensions import TypedDict
from typing import List

class GraphState(TypedDict):
    question: str
    profile: dict
    notes: List[str]
    web_result: str
    final_answer: str

def create_context(state):
    profile = state["profile"]
    notes = state["notes"]
    profile_str = "\n".join([f"{k}: {v}" for k, v in profile.items()])
    notes_str = "\n".join(notes) or "None"
    return f"User Profile:\n{profile_str}\n\nNotes:\n{notes_str}\n\nQuestion:\n{state['question']}"

def load_context(state):
    return {
        "question": state["question"],
        "profile": load_user_profile(),
        "notes": fetch_notes(state["question"])
    }

def route_to_web(state):
    context = create_context(state)
    return "web_search" if should_use_web(context) else "no_web"

def perform_web_search(state):
    return {"web_result": search_web(state["question"])}

def generate_answer(state):
    profile_str = "\n".join([f"{k}: {v}" for k, v in state["profile"].items()])
    notes_str = "\n".join(state["notes"])
    web_result = state.get("web_result", "")
    prompt = f"""
You are a helpful fitness coach. Answer the user query in under 120 words.

Profile:
{profile_str}

Notes:
{notes_str}

Web Info:
{web_result}

Question:
{state['question']}
"""
    response = llm.invoke(prompt)
    return {"final_answer": response.content.strip()}

workflow = StateGraph(GraphState)
workflow.add_node("load_context", load_context)
workflow.add_node("web_search", perform_web_search)
workflow.add_node("generate_answer", generate_answer)

workflow.set_entry_point("load_context")
workflow.add_conditional_edges("load_context", route_to_web, {
    "web_search": "web_search",
    "no_web": "generate_answer"
})
workflow.add_edge("web_search", "generate_answer")
workflow.add_edge("generate_answer", END)

fitness_app = workflow.compile()


In [ ]:
import gradio as gr

def update_profile_ui(age, height, weight, goal):
    update_user_profile({"age": age, "height": height, "weight": weight, "goal": goal})
    return "Profile updated."

def submit_question(user_question):
    state = {"question": user_question}
    for step in fitness_app.stream(state):
        if "generate_answer" in step:
            return step["generate_answer"]["final_answer"]
    return " Something went wrong."

def refresh_notes():
    choices, _ = get_notes_display_list()
    return gr.update(choices=choices)

with gr.Blocks() as demo:
    gr.Markdown("## 🏋️‍♂️ Fitness Assistant")

    with gr.Tab(" Profile Setup"):
        age = gr.Number(label="Age")
        height = gr.Number(label="Height (cm)")
        weight = gr.Number(label="Weight (kg)")
        goal = gr.Textbox(label="Fitness Goal (e.g., fat loss)")
        profile_btn = gr.Button("Update Profile")
        profile_output = gr.Textbox(label="Status")
        profile_btn.click(update_profile_ui, inputs=[age, height, weight, goal], outputs=profile_output)

    with gr.Tab("Add Notes"):
        note_input = gr.Textbox(label="Note")
        note_btn = gr.Button("Submit Note")
        note_output = gr.Textbox(label="Status")
        note_btn.click(add_note_to_vectorstore, inputs=note_input, outputs=note_output)

    with gr.Tab("Ask Questions"):
        question = gr.Textbox(label="Ask a fitness question")
        answer = gr.Textbox(label="Answer")
        question_btn = gr.Button("Submit")
        question_btn.click(submit_question, inputs=question, outputs=answer)

    with gr.Tab("View & Delete Notes"):
        notes_list = gr.CheckboxGroup(choices=[], label="Select Notes to Delete")
        refresh_btn = gr.Button("Refresh Notes")
        delete_btn = gr.Button("Delete Selected Notes")
        delete_output = gr.Textbox(label="Status")

        refresh_btn.click(fn=refresh_notes, outputs=[notes_list])
        delete_btn.click(fn=delete_selected_notes, inputs=[notes_list], outputs=[delete_output])

demo.launch(debug=True)


It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://901d252c2607a95516.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/gradio/queueing.py", line 625, in process_events
    response = await route_utils.call_process_api(
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/gradio/route_utils.py", line 322, in call_process_api
    output = await app.get_blocks().process_api(
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/gradio/blocks.py", line 2187, in process_api
    inputs = await self.preprocess_data(
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/gradio/blocks.py", line 1853, in preprocess_data
    processed_input.append(block.preprocess(inputs_cached))
                           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/gradio/components/checkboxgroup.py", line 130, in preprocess
    raise Error(
gradio.exceptions.Error: "Value: '1. I have injur